In [ ]:
import asyncio
from ipykernel.eventloops import register_integration
import ipywidgets as widgets
from IPython.display import display
from notebook_utils import NotebookUtils as book

In [ ]:
@register_integration('asyncio')
def loop_asyncio(kernel):
    '''Start a kernel with asyncio event loop support.'''
    loop = asyncio.get_event_loop()

    def kernel_handler():
        loop.call_soon(kernel.do_one_iteration)
        loop.call_later(kernel._poll_interval, kernel_handler)

    loop.call_soon(kernel_handler)
    try:
        if not loop.is_running():
            loop.run_forever()
    finally:
        loop.run_until_complete(loop.shutdown_asyncgens())
        loop.close()
        print('Stoping Loop')

In [ ]:
%gui asyncio

In [ ]:
def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

In [ ]:
slider = widgets.IntSlider()
text = widgets.Text()
accordion = widgets.Accordion(children=[slider, text])
accordion.set_title(0, 'Slider')
accordion.set_title(1, 'Text')

In [ ]:
async def f():
    times_called = 0
    # for i in range(10):
    while True:
        # print('did work %s'%times_called)
        x = await wait_for_change(slider, 'value')
        y = await wait_for_change(text, 'value')
        a = await wait_for_change(accordion, 'value')
        print('Slider: {}'.format(x))
        print('Text: {}'.format(y))
        times_called += 1
asyncio.ensure_future(f())
display(accordion, slider)

In [ ]:
print(slider.value)
print(text.value)

In [ ]:
backtester = widgets.Play(
    value = 0,
    min = 0,
    max = 100,
    step = 1,
    description = "Backtest",
    disabled = False
)
slid = widgets.IntSlider()
widgets.jslink((backtester, 'value'), (slid, 'value'))
row1 = widgets.HBox([backtester, slid])
row2 = widgets.HBox([text])
widgets.VBox([row1, row2])